# Second Agate Assignment

With this assignment, we need to import some data, filter out non-UNL campuses and calculate a average salary and a median salart for all UNL employees. The first step is always import the libraries you need. For now and the forseeable future, we just need Agate.

In [1]:
import agate

Let's add some data. 

In [2]:
salaries = agate.Table.from_csv('../../Data/nusalaries1415.csv')

In [3]:
print(salaries)

|-------------------+---------------|
|  column_names     | column_types  |
|-------------------+---------------|
|  ID               | Number        |
|  Campus           | Text          |
|  DepartmentNumber | Date          |
|  DepartmentName   | Text          |
|  CostElement      | Number        |
|  Name             | Text          |
|  Title            | Text          |
|  Position         | Number        |
|  Class            | Number        |
|  Term             | Number        |
|  FTE              | Number        |
|  Salary           | Number        |
|-------------------+---------------|



Now we just want UNL, so we need to filter those out. 

In [4]:
unl = salaries.where(lambda row: row['Campus'] is 'University of Nebraska-Lincoln')

In [5]:
print(len(unl.rows))

0


Uh oh. How could this be? The answer is almost always a bad filter condition. In this case, it's not title cased, it should be all caps. And there's a space between the dash on both sides.

In [6]:
unl = salaries.where(lambda row: row['Campus'] is 'UNIVERSITY OF NEBRASKA - LINCOLN')

In [7]:
print(len(unl.rows))

0


Now, what the hell? That should work, right? Well, not exactly. We need to set our row equal to UNIVERISTY ... and we can't use the regular `=` to do it. We need to use `==` which in Python is actually equal to. The single equal sign is for assigning varables. 

In [8]:
unl = salaries.where(lambda row: row['Campus'] == 'UNIVERSITY OF NEBRASKA - LINCOLN')

In [9]:
print(len(unl.rows))

6948


That's better. Now let's create an average salary.

In [10]:
average_salary = unl.aggregate(agate.Mean('Salary'))
print(average_salary)

71766.36931491076568796776051


So the average salary is \$71,766. Which might be influenced by the \$2.5 million the football coach makes, or the cool million the basketball coach makes. Maybe. Let's see by creating a median salary.

In [11]:
median_salary = unl.aggregate(agate.Median('Salary'))
print(median_salary)

54200


That's much more realistic. But it's also not representative of individual jobs. We'll be curious about those. So let's group them together and do the same aggregates on the grouped salaries.

In [12]:
by_position = unl.group_by('Title')

Now we create some aggregates of those. We can chain those together into a single table. This will give us a count, a median and an average all in the same table so we can compare.

In [13]:
jobs = by_position.aggregate([
    ('count', agate.Length()),
    ('median_salary', agate.Median('Salary')),
    ('average_salary', agate.Mean('Salary'))
])

Now that we have that, we want to sort it and then print it out so we can see it. 

In [14]:
sorted_jobs = jobs.order_by('count', reverse=True)
sorted_jobs.print_table(max_rows=50)

|----------------------------------+-------+---------------+---------------------------------|
|  Title                           | count | median_salary |                 average_salary  |
|----------------------------------+-------+---------------+---------------------------------|
|  Professor                       |   788 |     122,289.0 | 129,447.2131979695431472081218  |
|  Associate Professor             |   324 |      77,648.0 |  83,942.1049382716049382716049  |
|  Assistant Professor             |   184 |      71,050.0 |  79,873.3152173913043478260870  |
|  Custodian II                    |   181 |      23,893.0 |  24,498.8453038674033149171271  |
|  Asst Professor                  |   152 |      83,353.0 |  83,441.4605263157894736842105  |
|  Assoc Professor                 |   127 |      93,592.0 |  96,058.7716535433070866141732  |
|  Director                        |    86 |     142,127.5 | 139,503.9302325581395348837209  |
|  Office Assoc                    |    81 |      

Being a full professor is a pretty good gig. But notice something about the jobs? Associate professor vs assoc professor? Same with assistant vs asst? That's a problem for another day. 